In [1]:
!pip install -qq transformers[sentencepiece]==4.35.2 datasets==2.16.1 evaluate==0.4.1
!sudo apt-get install -y libomp-dev
!pip install -qq faiss-gpu
!pip install -U langchain-community

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
cudf 24.4.1 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.5.0 which is incompatible.
distributed 2024.1.1 requires dask==2024.1.1, but you have dask 2024.5.2 which is incompatible.
gc

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
import numpy as np
import collections
import torch
import faiss
import evaluate
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer
from tqdm.auto import tqdm
from transformers import BertModel, BertTokenizer
from datasets import Dataset, load_dataset
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


2024-07-13 12:00:21.694054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 12:00:21.694188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 12:00:21.820850: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
file_path = "../input/machine/machine_learning.pdf"

In [4]:
loader = PyPDFLoader(file_path = file_path)

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
)
data = loader.load_and_split(text_splitter = text_splitter)

In [18]:
data

[Document(metadata={'source': '../input/machine/machine_learning.pdf', 'page': 0}, page_content='Machine learning is a subset of artiﬁcial intelligence (AI) focused on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions, relying instead on patterns and inference derived from data. The concept of machine learning was ﬁrst introduced in the mid-20th century by Arthur Samuel, who coined the term "machine learning" in 1959. Samuel deﬁned it as a "ﬁeld of study that gives computers the ability to learn without being explicitly'),
 Document(metadata={'source': '../input/machine/machine_learning.pdf', 'page': 0}, page_content='programmed. "  The ﬁeld of machine learning has seen signiﬁcant evolution since its inception. In the 1980s and 1990s, the development of deep learning and artiﬁcial neural networks marked a major advancement. A pivotal moment came in 1986 when GeoNrey Hinton and his colleagues introduced the backprop

In [19]:
data[0].page_content

'Machine learning is a subset of artiﬁcial intelligence (AI) focused on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions, relying instead on patterns and inference derived from data. The concept of machine learning was ﬁrst introduced in the mid-20th century by Arthur Samuel, who coined the term "machine learning" in 1959. Samuel deﬁned it as a "ﬁeld of study that gives computers the ability to learn without being explicitly'

In [20]:
model = AutoModel.from_pretrained('distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')


In [21]:
def get_embeddings(document):
    inputs = tokenizer(document, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings


In [22]:
data[0]

Document(metadata={'source': '../input/machine/machine_learning.pdf', 'page': 0}, page_content='Machine learning is a subset of artiﬁcial intelligence (AI) focused on the development of algorithms and statistical models that enable computers to perform tasks without explicit instructions, relying instead on patterns and inference derived from data. The concept of machine learning was ﬁrst introduced in the mid-20th century by Arthur Samuel, who coined the term "machine learning" in 1959. Samuel deﬁned it as a "ﬁeld of study that gives computers the ability to learn without being explicitly')

In [23]:
#transform list to dict
texts = [doc.page_content for doc in data]
data_dict = {'context': texts}
raw_datasets = Dataset.from_dict(data_dict)

print(raw_datasets)

Dataset({
    features: ['context'],
    num_rows: 4
})


In [24]:
EMBEDDING_COLUMN = 'page_content_embedding'

embeddings_dataset = raw_datasets.map(
    lambda x: {EMBEDDING_COLUMN: get_embeddings(x['context']).detach().cpu().numpy()[0]}
)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [25]:
print(embeddings_dataset)

Dataset({
    features: ['context', 'page_content_embedding'],
    num_rows: 4
})


In [26]:
embeddings_dataset.add_faiss_index(column=EMBEDDING_COLUMN)

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['context', 'page_content_embedding'],
    num_rows: 4
})

In [29]:
input_question = 'When did backpropagation first appear?'

input_quest_embedding = get_embeddings([input_question]).cpu().detach().numpy()

TOP_K = 5
scores, samples = embeddings_dataset.get_nearest_examples(
    EMBEDDING_COLUMN, input_quest_embedding, k=TOP_K
)

for idx, score in enumerate(scores):
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Context: {samples["context"][idx]}')
    print()


Top 1	Score: 31.85853385925293
Context: programmed. "  The ﬁeld of machine learning has seen signiﬁcant evolution since its inception. In the 1980s and 1990s, the development of deep learning and artiﬁcial neural networks marked a major advancement. A pivotal moment came in 1986 when GeoNrey Hinton and his colleagues introduced the backpropagation algorithm, which became a fundamental technique for training neural networks.  In recent years, machine learning has experienced tremendous growth due to advancements in computational power

Top 2	Score: 33.920066833496094
Context: and the explosion of data. Modern applications of machine learning are diverse, encompassing areas such as speech recognition, image recognition, natural language processing, recommendation systems, and industrial automation.  Prominent ﬁgures like GeoNrey Hinton, Yann LeCun, and Andrew Ng have made signiﬁcant contributions to the ﬁeld, pushing the

Top 3	Score: 34.91851806640625
Context: Machine learning is a subs

In [30]:
PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'FuuToru/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)

# Use the results from the previous query (scores and samples)
print(f'Input question: {input_question}')
for idx, score in enumerate(scores):
    question = input_question
    context = samples["context"][idx]
    answer = pipe(
        question=question,
        context=context
    )
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Context: {context}')
    print(f'Answer: {answer["answer"]}')
    print()


Input question: When did backpropagation first appear?
Top 1	Score: 31.85853385925293
Context: programmed. "  The ﬁeld of machine learning has seen signiﬁcant evolution since its inception. In the 1980s and 1990s, the development of deep learning and artiﬁcial neural networks marked a major advancement. A pivotal moment came in 1986 when GeoNrey Hinton and his colleagues introduced the backpropagation algorithm, which became a fundamental technique for training neural networks.  In recent years, machine learning has experienced tremendous growth due to advancements in computational power
Answer: 1986

Top 2	Score: 33.920066833496094
Context: and the explosion of data. Modern applications of machine learning are diverse, encompassing areas such as speech recognition, image recognition, natural language processing, recommendation systems, and industrial automation.  Prominent ﬁgures like GeoNrey Hinton, Yann LeCun, and Andrew Ng have made signiﬁcant contributions to the ﬁeld, pushing the

In [35]:
class RAG:
    def __init__(self, document):
        self.document = document
        self.model = AutoModel.from_pretrained('distilbert-base-uncased')
        self.tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
    
    def load_document(self):
        loader = PyPDFLoader(file_path=self.document)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
        data = loader.load_and_split(text_splitter=text_splitter)
        texts = [doc.page_content for doc in data]
        data_dict = {'context': texts}
        raw_datasets = Dataset.from_dict(data_dict)
        return raw_datasets
    
    def get_embeddings(self, document):
        inputs = self.tokenizer(document, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings
    
    def compute_embeddings(self):
        raw_datasets = self.load_document()
        embeddings_dataset = raw_datasets.map(
            lambda x: {'page_content_embedding': self.get_embeddings(x['context']).detach().cpu().numpy()[0]}
        )
        embeddings_dataset.add_faiss_index(column='page_content_embedding')
        return embeddings_dataset


In [36]:
file_path = "../input/machine/machine_learning.pdf"

In [37]:
rag_instance = RAG(document=file_path)
embeddings_data = rag_instance.compute_embeddings()

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [41]:
question = "In what year was the backpropagation algorithm developed?"

In [42]:
input_question = question
input_question_embedding = rag_instance.get_embeddings(input_question).cpu().detach().numpy()

scores, samples = embeddings_data.get_nearest_examples(
    "page_content_embedding", input_question_embedding, k=1
)

PIPELINE_NAME = 'question-answering'
MODEL_NAME = 'FuuToru/distilbert-finetuned-squadv2'
pipe = pipeline(PIPELINE_NAME, model=MODEL_NAME)

print(f'Input question: {input_question}\n')
context = samples["context"][0]
answer = pipe(
    question=input_question,
    context=context
)
print(f'Context: {context}\n')
print(f'Answer: {answer["answer"]}')

Input question: In what year was the backpropagation algorithm developed?

Context: programmed. "  The ﬁeld of machine learning has seen signiﬁcant evolution since its inception. In the 1980s and 1990s, the development of deep learning and artiﬁcial neural networks marked a major advancement. A pivotal moment came in 1986 when GeoNrey Hinton and his colleagues introduced the backpropagation algorithm, which became a fundamental technique for training neural networks.  In recent years, machine learning has experienced tremendous growth due to advancements in computational power

Answer: 1986
